In [1]:
import json

In [2]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span

In [5]:
import json
from collections import defaultdict
from spacy.matcher import PhraseMatcher

In [9]:
import os
import glob

In [15]:
from collections import defaultdict

In [4]:
nlp = spacy.load('en_core_web_lg')

In [7]:
os.getcwd()

'/home/ec2-user/spacy-projects/tutorials/rel_component'

In [12]:
file_list = glob.glob("/home/ec2-user/ebs/ocr_data/12009/*.ocr")

In [14]:
len(file_list)

181

In [22]:
ocr_dict = defaultdict(list)
for filename in file_list:
    with open(filename, 'r') as fp:
        page_data = json.load(fp)
        blocks = page_data.get('Blocks', [])
        for block in blocks:
            if 'Text' in block:
                ocr_dict[filename].append({'text':block['Text'], 'geometry':block['Geometry']})

In [24]:
# ocr_dict['/home/ec2-user/ebs/ocr_data/12009/12009_dvm_dez_GenMono001490-00001-1.ocr']

In [25]:
os.getcwd()

'/home/ec2-user/spacy-projects/tutorials/rel_component'

In [27]:
os.chdir('/home/ec2-user/spacy-projects/tutorials/rel_component/scripts')

In [28]:
import random
import typer
from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example

# make the factory work
from rel_pipe import make_relation_extractor, score_relations

# make the config work
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [29]:
matcher_lemma = PhraseMatcher(nlp.vocab, attr='LEMMA')
matcher_tags = ["BIRTH",
"MARRIAGE",
"DEATH",
"SPOUSE"
"HUSBAND",
"WIFE",
"CHILD",
"SON",
"DAUGHTER",
"SIBLING",
"BROTHER",
"SISTER",
"PARENT",
"FATHER",
"MOTHER",
"GRANDPARENT",
"GRANDFATHER",
"GRANDMOTHER",
"GRANDCHILD",
"GRANDSON",
"GRANDDAUGHTER",
"COUSIN",
"AUNT",
"UNCLE",
"NIECE",
"NEPHEW"]

pattern_list = [nlp(text) for text in matcher_tags]
matcher_lemma.add("patterns", pattern_list)

In [31]:
for file_name, file_blocks in ocr_dict.items():
    for block in file_blocks:
        text = block['text']
        geometry = block['geometry']
        doc = nlp(text)
        matches = matcher_lemma(doc)
        possible_entity_label_list = []
        for match_id, start, end in matches:
            span = doc[start:end]
            possible_entity_label_list.append(span.text)
        block['possible_entity_labels'] = possible_entity_label_list

In [35]:
# ocr_dict['/home/ec2-user/ebs/ocr_data/12009/12009_dvm_dez_GenMono001490-00001-1.ocr'][10]

In [37]:
# for file_name, file_blocks in ocr_dict.items():
#     for block in file_blocks:
#         if block['possible_entity_labels']:
#             print(block['possible_entity_labels'])